<a href="https://colab.research.google.com/github/altra102/2D-Transformer/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/altra102/2D-Transformer.git
!pip install compress_pickle

Cloning into '2D-Transformer'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 71 (delta 31), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (71/71), done.


In [23]:
cd 2D-Transformer

/content/2D-Transformer


In [24]:
cd Model

/content/2D-Transformer/Model


In [25]:
from transformer import Transformer2D
from inference import inference

In [5]:
cd ..

/content/2D-Transformer


In [6]:
cd Training

/content/2D-Transformer/Training


In [9]:
from loadAndSafe import load_dict, save_dict
from iamDataset import dict_dataset, create_alphabet, create_converter
from train import LabelSmoothing, train, evaluate

In [10]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from time import time


In [ ]:
#IAM Train Demo

In [11]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oxNXBLmfZ0FiNSQM1q_xau_7AcnKnwQA' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1oxNXBLmfZ0FiNSQM1q_xau_7AcnKnwQA" -O iam_dict_1024_128_3_aachen.gz && rm -rf /tmp/cookies.txt 

--2021-10-07 15:25:26--  https://docs.google.com/uc?export=download&confirm=uo_T&id=1oxNXBLmfZ0FiNSQM1q_xau_7AcnKnwQA
Resolving docs.google.com (docs.google.com)... 173.194.76.138, 173.194.76.102, 173.194.76.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.76.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-6g-docs.googleusercontent.com/docs/securesc/7qkt3m3bdkvutmvekm6sbcov9bfgr237/f9krfbmlnaseast5r6s7qcmharrjvfme/1633620300000/11496534070352481065/11424203677153086091Z/1oxNXBLmfZ0FiNSQM1q_xau_7AcnKnwQA?e=download [following]
--2021-10-07 15:25:26--  https://doc-08-6g-docs.googleusercontent.com/docs/securesc/7qkt3m3bdkvutmvekm6sbcov9bfgr237/f9krfbmlnaseast5r6s7qcmharrjvfme/1633620300000/11496534070352481065/11424203677153086091Z/1oxNXBLmfZ0FiNSQM1q_xau_7AcnKnwQA?e=download
Resolving doc-08-6g-docs.googleusercontent.com (doc-08-6g-docs.googleusercontent.com)... 142.250.110.132, 2a00:1450:400c:c0d::84
Connectin

In [12]:
#setup dataset
iam_dict = load_dict('/content/2D-Transformer/Training/iam_dict_1024_128_3_aachen.gz')
alphabet = create_alphabet(iam_dict['train']['label'])
s2i, i2s = create_converter(alphabet)

b_size = 16
train_dataset = dict_dataset(iam_dict, s2i, 'train')
validation_dataset = dict_dataset(iam_dict, s2i, 'validation')

train_loader = DataLoader(train_dataset, b_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, b_size, shuffle=True)

In [ ]:
#setup model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classes = len(alphabet)
lr = 0.0002

model = Transformer2D(classes, 2048, 256, n_heads=2 , num_layers=1, cnn='resnet50', partial_cnn='layer4', height=8 , width=240, dropout=0.4, pretrain=True, onedim=True).to(device)
criterion = LabelSmoothing(size=classes, padding_idx=0, smoothing=0.8).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.75)            

#init param
for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

save_name = 'demo_save_name'
save_path = f"/content/{save_name}.pt"

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

best_validation = np.inf
c = 0
early_stop = 0
train_losses = []
val_losses = []
for epoch in range(5):  #change this to 1000 for better results 
 
    print(f'Epoch: {epoch+1:02}','learning rate{}'.format(scheduler.get_last_lr()))
    
    start_time = time()

    train_loss = train(model, classes, criterion, optimizer, scheduler, train_loader, device)
    validation_loss = evaluate(model, classes, criterion, validation_loader, device)

    epoch_mins, epoch_secs = epoch_time(start_time, time())
  
    c+=1
    if validation_loss < best_validation:
        best_validation = validation_loss
        torch.save(model.state_dict(), save_path)
        early_stop = 0
        c=0
    else:
        early_stop +=1
    if c>3:
        #decrease lr if loss does not deacrease after 3 steps
        scheduler.step()
        c=0
    if early_stop == 20: # 20 times bad eval = early stop
        break

    print(f'Time: {epoch_mins}m {epoch_secs}s') 
    print(f'Train Loss: {train_loss:.3f}, Val   Loss: {validation_loss:.3f}, best Val Loss: {best_validation:.3f}')

    train_losses.append(train_loss)
    val_losses.append(validation_loss)

#save losses
save_dict(list(zip(train_losses,val_losses)),"/content", save_name + '_loss')


Epoch: 01 learning rate[0.0002]


  0%|          | 0/412 [00:00<?, ?it/s]

Time: 2m 26s
Train Loss: 217.518, Val   Loss: 209.503, best Val Loss: 209.503
Epoch: 02 learning rate[0.0002]


  0%|          | 0/412 [00:00<?, ?it/s]

Time: 2m 25s
Train Loss: 210.378, Val   Loss: 203.864, best Val Loss: 203.864
Epoch: 03 learning rate[0.0002]


...

Time: 2m 25s
Train Loss: 205.582, Val   Loss: 197.413, best Val Loss: 197.413
Epoch: 04 learning rate[0.0002]


  0%|          | 0/412 [00:00<?, ?it/s]

Time: 2m 26s
Train Loss: 202.007, Val   Loss: 199.145, best Val Loss: 197.413
Epoch: 05 learning rate[0.0002]


  0%|          | 0/412 [00:00<?, ?it/s]

Time: 2m 25s
Train Loss: 199.296, Val   Loss: 193.211, best Val Loss: 193.211
